<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/ListaSeiRevisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install fuzzywuzzy
# !pip install levenshtein
# !pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 4.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

basedf = pd.read_excel('/content/Prof Pioneiros Referencia tudo.xlsx', sheet_name='285715-CNV')

/usr/local/lib/python3.8/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# basedf.info()

In [ ]:
#Adiciona uma cópia da coluna SIAPE
basedf.insert(11, 'IT-NO-SIAPE', basedf['Siape'])

#Altera o nome da coluna "Siape" para "SIAPE", dessa forma o nome ficará igual entre a basedf e baserq
basedf = basedf.rename(
    columns = {'Siape':'SIAPE'}
    )

#Concatena Cargo e Cargo.1 em IT-CO-CARGO-EMPREGO
basedf['IT-CO-CARGO-EMPREGO'] = basedf['Cargo'].astype(str) + ' - ' + basedf['Cargo.1'].astype(str)

In [161]:
#Lista SEI de requerimentos para selecionar 40801-AP 40803-RO 40804-RR

# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_COMPLEMENTO_40804_09_01_2023.xlsx')
baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_COMPLEMENTO_40801_09_01_2023.xlsx')
# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_COMPLEMENTO_40801_23_01_2023.xlsx')
# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_40801_26_12_2022.xlsx')
# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_40803_27_12_2022.xlsx', sheet_name='Ok')
# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_40804_27_12_2022.xlsx')

Original = baserq
Original['DATA DO REQUERIMENTO'] = pd.to_datetime(Original['DATA DO REQUERIMENTO'], errors="coerce")

In [162]:
#Renomeia as colunas para evitar conflitos no merge
baserq = baserq.rename(
    columns = {'DESCRIÇÃO':'ASSUNTO'}
    )

In [163]:
# baserq.info()

In [164]:
baserq['DATA DO REQUERIMENTO'] = pd.to_datetime(baserq['DATA DO REQUERIMENTO'], errors="coerce")

#Muda os caracteres para maiúsculos
baserq['INTERESSADO'] = baserq['INTERESSADO'].str.upper()
baserq['DATA DO REQUERIMENTO'] = baserq['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')

In [165]:
#Corrige os números SIAPE dos servidores MARIA JOSÉ BARBOSA DA SILVA, ANA HELENA FIGUEIREDO SILVA, FRANCISCO SILVA CUSTÓDIO e CIPRIANA DOS SANTOS MACEDO em 40801.

# Dicionário com as informações de mudança
substituicoes = {10109595:1009595, 
                 101117:1011117, 
                 101527:1011527, 
                 1111456:1011456}

#Substituindo
for valor_antigo, valor_novo in substituicoes.items():
    baserq.loc[baserq['SIAPE'] == valor_antigo, 'SIAPE'] = valor_novo

In [166]:
# #Corrige os números SIAPE dos servidores em 40804. 
# # JOSUÉ DA SILVA, OSBELTO RIBEIRO TRINDADE, SILVANA RODRIGUES DA SILVA, 
# # MIRIAN FERNANDES DE MELO, ANA AMELIA ALMEIDA SILVA, MARILENE EUSEBIO TOME MUNHOZ, 
# # ELANI DO SOCORRO FERNANDES PINHEIRO STUART, MARLEDE C.SCHWAB e ROSANGELA DA COSTA WERMOUTH 

# # Dicionário com as informações de mudança
# substituicoes = {700955:709955, 
#                  1149046:707477, 
#                  6713577:713577, 
#                  7058489:705848,
#                  7095619:709561, 
#                  7115326:711532, 
#                  7135882:713588, 
#                  10365133:1036513, 
#                  1148996:707406}

# #Substituindo
# for valor_antigo, valor_novo in substituicoes.items():
#     baserq.loc[baserq['SIAPE'] == valor_antigo, 'SIAPE'] = valor_novo

In [167]:
# #Corrige os números SIAPE dos servidores EUGENIA MARIA FONSECA DOS REIS em 40801.

# # Dicionário com as informações de mudança
# substituicoes = {1030350:1010350}

# #Substituindo
# for valor_antigo, valor_novo in substituicoes.items():
#     baserq.loc[baserq['SIAPE'] == valor_antigo, 'SIAPE'] = valor_novo

In [168]:
# Junta as planilhas basedf e baserq pra identificar correspondência de SIAPE
mixResultadoGeral = baserq.merge(
    basedf, on=['SIAPE'], 
    how='outer', 
    # how='right', 
    suffixes=['', '_'], 
    indicator=True
    )

# Adiciona uma coluna com legendas para cada situação
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only'), 'CONDIÇÃO']   = 'Número SIAPE não encontrado em Prof Pioneiros Referencia tudo'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both'), 'CONDIÇÃO']        = 'Número SIAPE encontrado em Prof Pioneiros Referencia tudo'


# Lista de colunas para manter
colunas_manter = ['PROCESSO SEI', 
                  'DATA DO REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'ASSUNTO',
                  'GR-MATRICULA',
                  'Órgão',
                  'IT-NU-IDEN-SERV-ORIGEM',
                  'IT-NO-SIAPE',
                  'IT-NO-SERVIDOR',
                  'IT-NU-CPF',
                  '_merge',
                  'IT-CO-CARGO-EMPREGO']

# Exclusão das colunas que não estão na lista
mixResultadoGeral = mixResultadoGeral[colunas_manter]


#Calcula a similaridade entre os nomes dos interessados e dos servidores usando cálculo de distância de Levenshtein. Indentificar se o servidor foi o requerente.
mixResultadoGeral['SIMILARIDADE'] = mixResultadoGeral.apply(lambda row: fuzz.partial_ratio(str(row['INTERESSADO']), str(row['IT-NO-SERVIDOR'])) if (isinstance(row['INTERESSADO'],str) and isinstance(row['IT-NO-SERVIDOR'],str)) else None, axis=1)


In [169]:
colunas_manter_sim = ['PROCESSO SEI', 
                  'DATA DO REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'ASSUNTO',
                  'GR-MATRICULA',
                  'Órgão',
                  'IT-NU-IDEN-SERV-ORIGEM',
                  'IT-NO-SIAPE',
                  'IT-NO-SERVIDOR',
                  'IT-NU-CPF', 
                  '_merge',
                  'IT-CO-CARGO-EMPREGO',
                  'SIMILARIDADE']

SimilaridadeMenor50   = mixResultadoGeral.loc[(mixResultadoGeral['SIMILARIDADE'] <= 50)]
SimilaridadeMaior50   = mixResultadoGeral.loc[(mixResultadoGeral['SIMILARIDADE'] > 50)]
SimilaridadeMenor50   = SimilaridadeMenor50[colunas_manter_sim]
SimilaridadeMaior50   = SimilaridadeMaior50[colunas_manter_sim]

In [170]:
# Cria uma nova planilha somente com os servidores não encontrados na Prof Pioneiros Referencia tudo
servNao = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only')]
# servNao = SimilaridadeMenor50.loc[(SimilaridadeMenor50['_merge'] == 'both')]

# Cria uma nova planilha somente com os servidores encontrados Prof Pioneiros Referencia tudo
# servEnc = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both')]
servEncMaior = SimilaridadeMaior50.loc[(SimilaridadeMaior50['_merge'] == 'both')]
servEncMenor = SimilaridadeMenor50.loc[(SimilaridadeMenor50['_merge'] == 'both')]
#Gerando as planilhas de Encontrados e Não Encontrados
now = datetime.now()

if not servNao.empty:
    servNaoEnc = f"servNao{now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
    servNao.to_excel(servNaoEnc, index=False)
else:
    print("O DataFrame servNao está vazio, arquivo não será criado.")

O DataFrame servNao está vazio, arquivo não será criado.


In [171]:
#Exclue a coluna _merge
servEncMaior.drop('_merge', axis = 1, inplace=True)
servEncMenor.drop('_merge', axis = 1, inplace=True)

In [172]:
#Agrupa as primeiras ocorrências dos SIAPEs alvos
resultadoMaior = servEncMaior.groupby('SIAPE').first().reset_index()
resultadoMenor = servEncMenor.groupby('SIAPE').first().reset_index()

#Ajusta o formato da data para dd/mm/aaaa
resultadoMaior['DATA DO REQUERIMENTO'] = pd.to_datetime(resultadoMaior['DATA DO REQUERIMENTO'], errors="coerce")
resultadoMaior['DATA DO REQUERIMENTO'] = resultadoMaior['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')
resultadoMenor['DATA DO REQUERIMENTO'] = pd.to_datetime(resultadoMenor['DATA DO REQUERIMENTO'], errors="coerce")
resultadoMenor['DATA DO REQUERIMENTO'] = resultadoMenor['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')
Original['DATA DO REQUERIMENTO']  = Original['DATA DO REQUERIMENTO'].dt.strftime('%d/%m/%Y')

#colunas
cols = resultadoMaior.columns.tolist()
colsmenor = resultadoMenor.columns.tolist()

#colocando SIAPE na posição 2
cols.insert(2, cols.pop(cols.index('SIAPE')))
colsmenor.insert(2, colsmenor.pop(colsmenor.index('SIAPE')))

#atribuindo as novas colunas
resultadoMaior = resultadoMaior[cols]
resultadoMenor = resultadoMenor[colsmenor]


#Gerando a planilha ResultadoRevisao
# now = datetime.now()
# ResultadoRevisao = f"ResultadoRevisao{now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
# resultado.to_excel(ResultadoRevisao, index=False)

# Usando o ExcelWriter, cria um doc .xlsx, usando engine='xlsxwriter'
ResultadoRevisao = f"ResultadoRevisao{now.strftime('%Y-%m-%d %H:%M:%S')}.xlsx"
writer = pd.ExcelWriter(ResultadoRevisao, engine='xlsxwriter')

# Armazena cada df em uma planilha diferente do mesmo arquivo
Original.to_excel(writer, sheet_name='AP_ORIGINAL')
resultadoMaior.to_excel(writer, sheet_name='AP_TRATADA')
resultadoMenor.to_excel(writer, sheet_name='AP_PEDENCIAS')


# Fecha o ExcelWriter e gera o arquivo .xlsx
writer.save()
